# 2. Constant Memory


Constant Memory는 Read-only Cache 메모리입니다. 64KB 밖에 되지는 않지만 CUDA block간에 모두 공유가 되어서 쉽게 이용할 수 있습니다.

다만 이번 예제에서는 SGEMM이 아닌 다른 예제를 활용해서 속도를 비교하도록 하겠습니다.

In [1]:
%%file constant.cu

#include <stdio.h>
#include <stdlib.h>
#include <math_constants.h>

#define BLOCK_DIM 16
#define PI_STEP 256

__constant__ float c_sin_table[PI_STEP * 2];
__constant__ float c_cos_table[PI_STEP * 2];

__global__ void foo(float* out, const int width, const int height) {
    int idx_x = blockDim.x * blockIdx.x + threadIdx.x;
    int idx_y = blockDim.y * blockIdx.y + threadIdx.y;
    int idx = idx_y * width + idx_x;
    
    if (idx_x < width && idx_y < height) {
        
        out[idx] = sin(idx * CUDART_PI_F / PI_STEP) * sin(idx * CUDART_PI_F / PI_STEP)
                 - cos(idx * CUDART_PI_F / PI_STEP) * cos(idx * CUDART_PI_F / PI_STEP);
    }
}

__global__ void fooConstant(float* out, const int width, const int height) {
    int idx_x = blockDim.x * blockIdx.x + threadIdx.x;
    int idx_y = blockDim.y * blockIdx.y + threadIdx.y;
    int idx = idx_y * width + idx_x;
    
    if (idx_x < width && idx_y < height) {
        out[idx] = c_sin_table[idx % (PI_STEP * 2)] * c_sin_table[idx % (PI_STEP * 2)]
                 - c_cos_table[idx % (PI_STEP * 2)] * c_cos_table[idx % (PI_STEP * 2)];
    }
}

int main() {
    int width = 4096;
    int height = 4096;
    float sin_table[PI_STEP * 2];
    float cos_table[PI_STEP * 2];
    float *first, *second;
    float *d_first, *d_second;
    
    // CUDA Event Create to estimate elased time
    cudaEvent_t start, stop;
    float elapsed_gpu;
    
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    for (int i = 0; i < PI_STEP * 2; i++) {
        sin_table[i] = sin(i * CUDART_PI_F / PI_STEP);
        cos_table[i] = cos(i * CUDART_PI_F / PI_STEP);
    }
    
    first = (float*)malloc(width * height * sizeof(float));
    second = (float*)malloc(width * height * sizeof(float));
    cudaMalloc((void**)&d_first, width * height * sizeof(float));
    cudaMalloc((void**)&d_second, width * height * sizeof(float));
    
    // Copy to Constant Memory
    cudaMemcpyToSymbol(c_sin_table, sin_table, PI_STEP * 2 * sizeof(float));
    cudaMemcpyToSymbol(c_cos_table, cos_table, PI_STEP * 2 * sizeof(float));
    
    dim3 blockDim(BLOCK_DIM, BLOCK_DIM);
    dim3 gridDim((width + BLOCK_DIM - 1) / BLOCK_DIM, (height + BLOCK_DIM - 1) / BLOCK_DIM);
    
    cudaEventRecord(start, 0);
    foo<<<gridDim, blockDim>>>(d_first, width, height);
    
    // Estimate CUDA operation time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    
    cudaEventElapsedTime(&elapsed_gpu, start, stop);
    printf("SGEMM CUDA Elapsed time (operation): %f ms\n", elapsed_gpu);
        
    cudaEventRecord(start, 0);
    fooConstant<<<gridDim, blockDim>>>(d_second, width, height);
    
    // Estimate CUDA operation time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    
    cudaEventElapsedTime(&elapsed_gpu, start, stop);
    printf("SGEMM CUDA Elapsed time (constant): %f ms\n", elapsed_gpu);
    
    cudaMemcpy(first, d_first, width * height * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(second, d_second, width * height * sizeof(float), cudaMemcpyDeviceToHost);
    
    float diff_sum = 0.f;
    for (int i = 0; i < width * height; i++) {
        if (first[i] - second[i] != 0.f) {
            diff_sum += first[i] - second[i];
        }
    }
    printf("diff_sum: %f\n", diff_sum);
    
    cudaFree(d_first);
    cudaFree(d_second);
    free(first);
    free(second);
    
    // finalize CUDA event
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

Overwriting constant.cu


In [2]:
! make constant

nvcc --ptxas-options=--verbose -gencode arch=compute_30,code=sm_30 -I/usr/local/cuda/samples/common/inc constant.cu -o constant 
ptxas info    : 0 bytes gmem, 4120 bytes cmem[3]
ptxas info    : Compiling entry function '_Z11fooConstantPfii' for 'sm_30'
ptxas info    : Function properties for _Z11fooConstantPfii
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 8 registers, 336 bytes cmem[0]
ptxas info    : Compiling entry function '_Z3fooPfii' for 'sm_30'
ptxas info    : Function properties for _Z3fooPfii
    32 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 14 registers, 336 bytes cmem[0], 48 bytes cmem[2]


In [3]:
! ./constant

SGEMM CUDA Elapsed time (operation): 15.945632 ms
SGEMM CUDA Elapsed time (constant): 9.857184 ms
diff_sum: -0.066286


위 예제를 통해서 Constant Memory를 사용할 경우, 사용하지 않은 경우에 비해서 성능 향상을 얻을 수 있었습니다.

위 예제의 경우처럼, constant memory는 look-up table을 구성해서 사용하는데 이점이 있습니다. 하지만 look-up table을 이용했을 때 단점인 연산오차에 대해 감안하셔야 합니다. 개발하시는 application의 오차 허용 범위가 look-up table을 사용한 경우와, look-up table이 이미 있고 Constant memory의 크기보다 작은 경우 사용해 보실 것을 권해 드립니다.